In [23]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from pororo import Pororo

In [46]:
# 영상의 url 입력
url = 'https://www.youtube.com/watch?v=PrafBZ5ca6Y'

In [47]:
# 영상 url로부터 댓글을 크롤링해 csv파일로 저장하는 함수
def video_comment_crawling(save_csv=True):
    comment_list = []

    driver = webdriver.Chrome('./chromedriver.exe')
    driver.maximize_window()
    
    
    print(f'Start comment crawling')
    
    driver.get(url)  # url에 접속
    time.sleep(3)

    count = 0
    body = driver.find_element_by_tag_name("body")

    # 댓글 데이터를 가져옴
    last = driver.find_elements_by_css_selector('#content-text')

    while True:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.4)

        new = driver.find_elements_by_css_selector('#content-text') # 댓글 내용

        if new == last:
            if count == 10:
                break
            count += 1
        else:
            count = 0

        last = new

    for idx in new:
        # 한글 깨짐 방지
        text = idx.text

        for idx in range(len(text)):
            if not ((0 <= ord(text[idx]) < 128) or (0xac00 <= ord(text[idx]) <= 0xd7af)):
                text = text.replace(text[idx], ' ')

        text = text.replace('\r\n', ' ')
        comment_list.append([text])
            
    driver.close()
    print('Finish comment crawling')
    
    
    if save_csv == True:
        # 댓글 데이터를 csv 파일에 저장
        dataframe = pd.DataFrame(comment_list, columns=["comment"])
        dataframe.to_csv('./youtube_comment.csv', mode = 'w', encoding='cp949')
        print('Saving data to csv file')
        
    return comment_list

In [48]:
def sentiment_analysis(comment_list):
    sa = Pororo(task='sentiment', lang='ko') # 감성 분석 task 모델

    pos_prob, neg_prob = 0.0, 0.0
    for comment in comment_list:
        sa_result = sa(comment[0], show_probs=True)

        pos_prob += sa_result['positive']
        neg_prob += sa_result['negative']

    total_num = len(comment_list) # 크롤링한 댓글 수
    pos_prob, neg_prob = pos_prob/total_num, neg_prob/total_num
    pos_perc, neg_perc = round(pos_prob*100), round(neg_prob*100)

    print('##### 댓글 선호도 분석 결과 #####')
    print(f'긍정: {pos_perc}%, 부정: {neg_perc}%')

In [ ]:
if __name__ == "__main__":
    comment_list = video_comment_crawling(save_csv=True)
    sentiment_analysis(comment_list)